<h1><center>Can i use your (fast) computer??</center></h1>
<h1><center>A Loop that runs for days</center></h1>
<h2><center>Investment Scenarios : 1,054,550 - Estimated Run Time in Hours:  328.08</center></h2>
<h1><center>Rodrigo Medina</center></h1>
<h1><center>November 2022</center></h1>
<br>

# Index


### 1.Methodology
***

### 2.Implementation
***


<br>
<br>

In [ ]:
from IPython.display import display

<h1><center> 1. Methodology</center></h1>

<center> The loop at the end of this code will evaluate investing in 50 different time lenght periods in 1,128 different <br><br>starting dates grouping industries into 24 different groups, the possible combinations using date restrictions<br><br> according to the dataset are 1,022,400 </center>
<br><br>

Calculating Loop Permutations:


In [174]:
investment_scenarios = 0

for h in range(1,24):
    bin_quantity = h
    for i in range(1,56):
        for k in range(0,100,1):
                
                    end_date_shift = k
                    end_date = 202313 - (end_date_shift*100)
                    investment_time_in_years = i
                    start_date = end_date - (investment_time_in_years*100)
                    #202112
                    for x in range(0,12):
                        start_date = start_date -  1
                       
                        last_two_sd = int(str(start_date)[-2:])
                        end_date = start_date + (investment_time_in_years*100)
                                             
                         
                        if start_date > 202108:
                         continue
                        if last_two_sd > 13:
                         continue
                        if last_two_sd == 0:
                         continue
                        if start_date < 192607:
                         continue 

                        #print('start date ' , start_date, 'end date ', end_date , 'inv period' , i , 'date dif' , (end_date - start_date), 'k ' ,k , 'bin' ,h ,'x=',x)
                        investment_scenarios += 1


In [175]:

Estimated_secs_per_scenario = 1.12 #prior time estimate per scenario
print('Investment Scenarios for all bins: ' + str(investment_scenarios) + ' - Estimated Run Time in Hours: ' , (round((investment_scenarios*Estimated_secs_per_scenario)/3600,2)))
            

Investment Scenarios for all bins: 1054550 - Estimated Run Time in Hours:  328.08


<h1><center> 2. Implementation</center></h1>


In [132]:
import pandas as pd
pd.set_option('display.max_rows', 210)
import yfinance as yf 
import matplotlib.pyplot as plt
import seaborn as sns 
import statsmodels.formula.api as smf 
import scipy.stats as stats
import numpy as np
import datetime
import plotly.express as px
import time

In [133]:
# Reading Fama and French Datasets
DF_Monthly_Rets_ValW = pd.read_csv('C:/Users/rodri/Desktop/RUNIM5ML/FPCS/Data/DF1 - 49_Ind_P_Monthly_Rets_Val_W.csv',skiprows=0)
DF_Monthly_Rets_EqualW = pd.read_csv('C:/Users/rodri/Desktop/RUNIM5ML/FPCS/Data/DF2 - 49_Ind_P_Monthly_Rets_EQ_W.csv',skiprows=0)
DF_Monthly_FirmsbyIndustry = pd.read_csv('C:/Users/rodri/Desktop/RUNIM5ML/FPCS/Data/DF3 - 49_Ind_P_Monthly_Number_of_Firms.csv',skiprows=0)
DF_Monthly_Avg_Firm_Size = pd.read_csv('C:/Users/rodri/Desktop/RUNIM5ML/FPCS/Data/DF4 - 49_Ind_P_Monthly_Avg_Firm_Size.csv',skiprows=0)

In [134]:
# Creating Benchmarks for both Value Weighted and Equal Weighted returns data frames. 
# Benchmark will be the mean of all industries for each date
# Removing date column from returns data frame 
DF_Monthly_Rets_ValW_2 = DF_Monthly_Rets_ValW.drop(columns=['Date'])
DF_Monthly_Rets_EqualW_2 = DF_Monthly_Rets_EqualW.drop(columns=['Date'])

In [135]:
# Calculating the mean of all industries for a given period but not taking into
#   Account industries in which unaccouted values are expressed as -99.99
DF_Monthly_Rets_ValW_BM = pd.DataFrame()
DF_Monthly_Rets_EqualW_BM = pd.DataFrame()

#Value Weighted Benchmark
DF_Monthly_Rets_ValW_BM['Date'] = DF_Monthly_Rets_ValW['Date']
DF_Monthly_Rets_ValW_BM['Date FF'] = DF_Monthly_Rets_ValW['Date']
DF_Monthly_Rets_ValW_BM['BenchMark_ValW'] = DF_Monthly_Rets_ValW_2.where(DF_Monthly_Rets_ValW_2 != -99.99).mean(axis=1)
DF_Monthly_Rets_ValW_BM['Date'] = pd.to_datetime(DF_Monthly_Rets_ValW_BM['Date FF'], format="%Y%m") 
DF_Monthly_Rets_ValW_BM['Date'] = DF_Monthly_Rets_ValW_BM['Date'].dt.date 
DF_Monthly_Rets_ValW_BM.index = DF_Monthly_Rets_ValW_BM['Date']
DF_Monthly_Rets_ValW_BM = DF_Monthly_Rets_ValW_BM.drop(columns=['Date'])

#Equal Weighted Benchmark
DF_Monthly_Rets_EqualW_BM['Date'] = DF_Monthly_Rets_EqualW['Date']
DF_Monthly_Rets_EqualW_BM['Date FF'] = DF_Monthly_Rets_EqualW['Date']
DF_Monthly_Rets_EqualW_BM['BenchMark_EqualW'] = DF_Monthly_Rets_EqualW_2.where(DF_Monthly_Rets_EqualW_2 != -99.99).mean(axis=1)
DF_Monthly_Rets_EqualW_BM['Date'] = pd.to_datetime(DF_Monthly_Rets_EqualW_BM['Date FF'], format="%Y%m") 
DF_Monthly_Rets_EqualW_BM['Date'] = DF_Monthly_Rets_EqualW_BM['Date'].dt.date 
DF_Monthly_Rets_EqualW_BM.index =DF_Monthly_Rets_EqualW_BM['Date']
DF_Monthly_Rets_EqualW_BM = DF_Monthly_Rets_EqualW_BM.drop(columns=['Date'])

In [136]:
# Benchmark now is shifted one period ahead. Since the benchmark selects all industries invest in them 
# in the current period and accrues the return in next period.

DF_Monthly_Rets_ValW_BM['BenchMark_ValW_Lead_1'] = DF_Monthly_Rets_ValW_BM['BenchMark_ValW'].shift(-1)
DF_Monthly_Rets_EqualW_BM['BenchMark_EqualW_Lead_1'] = DF_Monthly_Rets_EqualW_BM['BenchMark_EqualW'].shift(-1)


In [137]:
# Creating an Industry Name Dictionary 
industry_name_dict = dict()
for i in DF_Monthly_Rets_ValW:
      if i != 'Date':
           industry_name_dict[i] = i

In [138]:
# Creating an Industry Dataframe for each industry combaning information of the 4 initial Fama and French Datasets
industry_dict = dict()
#%% Adding Lead variables, and benchmark

    
for i in industry_name_dict:  
    df = pd.DataFrame()
    df['Date FF'] = DF_Monthly_Rets_ValW['Date']
    df['Indsury'] = i
    df['Firms by Industry'] = DF_Monthly_FirmsbyIndustry[i]
    df['Avg Firm Size'] = DF_Monthly_Avg_Firm_Size[i]
    df['Ret_ValW'] = DF_Monthly_Rets_ValW[i]
    df['Ret_ValW_Lead1'] = DF_Monthly_Rets_ValW[i].shift(-1)
    df['Ret_EqualW'] = DF_Monthly_Rets_EqualW[i]
    df['Ret_EqualW_lead1'] = DF_Monthly_Rets_EqualW[i].shift(-1)
    industry_dict[i] = df
    

In [139]:
#Making a single Data Frame with all industries for Cross Sectional Analysis
Ind_Cross_Section_DF = pd.DataFrame()
df2 = pd.DataFrame()
for i in industry_dict.items():
            df2 = i[1]
            Ind_Cross_Section_DF =Ind_Cross_Section_DF.append(df2) 

In [140]:
#Date transformatin from string or int to datetime
Ind_Cross_Section_DF['Date'] = pd.to_datetime(Ind_Cross_Section_DF['Date FF'], format="%Y%m") 
Ind_Cross_Section_DF['Date'] = Ind_Cross_Section_DF['Date'].dt.date 

In [141]:
#  Sort by Date
Ind_Cross_Section_DF = Ind_Cross_Section_DF.sort_values(by=['Date'])

In [142]:
# Removing Nan Values - Fama and French database expresses them as -99.99
Ind_Cross_Section_DF = Ind_Cross_Section_DF.loc[Ind_Cross_Section_DF["Ret_ValW"] != -99.99 ]
Ind_Cross_Section_DF = Ind_Cross_Section_DF.loc[Ind_Cross_Section_DF["Ret_EqualW"] != -99.99 ]
Ind_Cross_Section_DF = Ind_Cross_Section_DF.loc[Ind_Cross_Section_DF["Avg Firm Size"] != -99.99 ]

In [143]:
# The full data set has a range from July 1926 to August 2022

<h2><center> Analysis in Different Time Periods 1 Million Scenarios - Full Sample Period 1926-2022</center></h2>



In [168]:
data1 = [] 
data2 = []
data3 = []
data4 = []
data5 = []
data6 = []
data7 = []
data8 = []
data9 = []
data10 = []
data11 = []
data12 = []
data13 = []
data14 = []
data15 = []
data16 = []
data17 = []
data18 = []
data19 = []




investment_scenarios = 0

start_time = time.time()

for h in range(1,24):
    bin_quantity = h
    for i in range(1,56):
        for k in range(0,100,1):
                
                    
                    Ind_Cross_Section_DF = pd.DataFrame()
                    df2 = pd.DataFrame()
                    for j in industry_dict.items():
                        df2 = j[1]
                        Ind_Cross_Section_DF =Ind_Cross_Section_DF.append(df2) 
                    Ind_Cross_Section_DF['Date'] = pd.to_datetime(Ind_Cross_Section_DF['Date FF'], format="%Y%m") 
                    Ind_Cross_Section_DF['Date'] = Ind_Cross_Section_DF['Date'].dt.date       
                    Ind_Cross_Section_DF = Ind_Cross_Section_DF.sort_values(by=['Date'])
                    Ind_Cross_Section_DF = Ind_Cross_Section_DF.loc[Ind_Cross_Section_DF["Ret_ValW"] != -99.99 ]
                    Ind_Cross_Section_DF = Ind_Cross_Section_DF.loc[Ind_Cross_Section_DF["Ret_EqualW"] != -99.99 ]
                    Ind_Cross_Section_DF = Ind_Cross_Section_DF.loc[Ind_Cross_Section_DF["Avg Firm Size"] != -99.99 ]

                
                    end_date_shift = k
                    end_date = 202313 - (end_date_shift*100)
                    investment_time_in_years = i
                    start_date = end_date - (investment_time_in_years*100)
                    #202112
                    for x in range(0,12):
                        start_date = start_date -  1
                       
                        last_two_sd = int(str(start_date)[-2:])
                        end_date = start_date + (investment_time_in_years*100)
                        
                        if start_date > 202108:
                         continue
                        if last_two_sd > 13:
                         continue
                        if last_two_sd == 0:
                         continue
                        if start_date < 192607:
                         continue
                        
                        Ind_Cross_Section_DF =  Ind_Cross_Section_DF[(Ind_Cross_Section_DF["Date FF"] > start_date) & (Ind_Cross_Section_DF["Date FF"] <= end_date)]

                        print('start date ' , start_date, 'end date ', end_date , 'inv period' , i , 'date dif' , (end_date - start_date), 'k ' ,k , 'bin' ,h ,'x=',x)
                        investment_scenarios += 1

                        Ind_Cross_Section_DF['Group_Firms by Industry'] = Ind_Cross_Section_DF.groupby('Date')['Firms by Industry'].transform(lambda x: pd.qcut(x, bin_quantity, labels= False, duplicates = 'drop')) # 
                        # Creating a return dataframe returns are shifted one period ahead.
                        firms_by_industry_q_rets_ValueW = pd.pivot_table(Ind_Cross_Section_DF, values='Ret_ValW_Lead1',index='Date',columns='Group_Firms by Industry',aggfunc=np.mean, fill_value=0)
                        firms_by_industry_q_rets_ValueW['Long Strategy Ret'] = firms_by_industry_q_rets_ValueW[0.0]/100
                        firms_by_industry_q_rets_ValueW['Long Strategy Cumm Ret'] = firms_by_industry_q_rets_ValueW['Long Strategy Ret'].add(1).cumprod()
                        #Adding the Benchmark
                        df_firms_by_industry_q_rets_ValueW = pd.merge(firms_by_industry_q_rets_ValueW ,DF_Monthly_Rets_ValW_BM['BenchMark_ValW_Lead_1'].divide(100),left_index=True,right_index=True,how='left')
                        # Adding the cummulative benchmark return
                        df_firms_by_industry_q_rets_ValueW['BenchMark_ValW_Cumm_Ret_Lead_1'] = df_firms_by_industry_q_rets_ValueW['BenchMark_ValW_Lead_1'].add(1).cumprod()
                        #Creating Results Dataframe
                        data1.append(bin_quantity)
                        data2.append(firms_by_industry_q_rets_ValueW['Long Strategy Ret'].mean()*12) 
                        data3.append(firms_by_industry_q_rets_ValueW['Long Strategy Ret'].std()*(12**.5))
                        data4.append((firms_by_industry_q_rets_ValueW['Long Strategy Ret'].mean()*12)/(firms_by_industry_q_rets_ValueW['Long Strategy Ret'].std()*(12**.5)))
                        data5.append(df_firms_by_industry_q_rets_ValueW['BenchMark_ValW_Lead_1'].mean()*12) 
                        data6.append(df_firms_by_industry_q_rets_ValueW['BenchMark_ValW_Lead_1'].std()*(12**.5)) 
                        data7.append((df_firms_by_industry_q_rets_ValueW['BenchMark_ValW_Lead_1'].mean()*12)/(df_firms_by_industry_q_rets_ValueW['BenchMark_ValW_Lead_1'].std()*(12**.5)))
                        data8.append(firms_by_industry_q_rets_ValueW['Long Strategy Cumm Ret'].iat[-1])
                        data9.append(df_firms_by_industry_q_rets_ValueW['BenchMark_ValW_Cumm_Ret_Lead_1'].iat[-1])
                        data10.append(start_date)
                        data11.append(investment_time_in_years)
                        data12.append(end_date)
                        data13.append((firms_by_industry_q_rets_ValueW['Long Strategy Cumm Ret'].iat[-1])/(df_firms_by_industry_q_rets_ValueW['BenchMark_ValW_Cumm_Ret_Lead_1'].iat[-1])-1)


                        number_of_bins = pd.DataFrame(data1, columns=['Number of bins / Ranking Categories'])
                        Long_strat_annual_return  = pd.DataFrame(data2,  columns=['Long Strategy Mean Annual Ret'])    
                        Long_strat_standard_deviation  = pd.DataFrame(data3,  columns=['Long Strategy Standard Deviation'])    
                        Long_strat_Return_Risk_Ratio = pd.DataFrame(data4,  columns=['Long Strategy Return Risk Ratio'])    
                        BM_mean_annual_return  = pd.DataFrame(data5,  columns=['Benchmark Annual Ret'])    
                        BM_standard_deviation  = pd.DataFrame(data6,  columns=['Benchmark Standard Deviation Ret']) 
                        BM_Return_Risk_Ratio = pd.DataFrame(data7,  columns=['Benchmark Return Risk Ratio']) 
                        Strat_Return_from_first_to_last_period =     pd.DataFrame(data8,  columns=['Strategy Total Cummulative Return']) 
                        BM_Return_from_first_to_last_period =     pd.DataFrame(data9,  columns=['BenchMark Total Cummulative Return'])

                        start_date_df =     pd.DataFrame(data10,  columns=['Investment Start Date']) 
                        start_date_df['Investment Start Date'] = pd.to_datetime(start_date_df['Investment Start Date'], format="%Y%m") 
                        investment_years_df =     pd.DataFrame(data11,  columns=['Investment Period in Years']) 
                        end_date_df =     pd.DataFrame(data12,  columns=['Investment End Date']) 
                        end_date_df['Investment End Date'] = pd.to_datetime(end_date_df['Investment End Date'], format="%Y%m") 
                        strat_performance_vs_benchmark =  pd.DataFrame(data13,  columns=['Strategy Return Performance vs Benchmark Return'])
                        strat_performance_vs_benchmark['Strategy Return Performance vs Benchmark Return'] = strat_performance_vs_benchmark['Strategy Return Performance vs Benchmark Return'].astype(float).map("{:.2%}".format)
                        Strategy_returns_table = pd.concat([number_of_bins,start_date_df,end_date_df,investment_years_df,strat_performance_vs_benchmark ,Strat_Return_from_first_to_last_period,Long_strat_annual_return,
                                                    Long_strat_standard_deviation,Long_strat_Return_Risk_Ratio,BM_Return_from_first_to_last_period,
                                                    BM_mean_annual_return,BM_standard_deviation,BM_Return_Risk_Ratio  ], axis=1)


    
    data14.append('Categorizing industries into ' + str(bin_quantity) + ' bins/categories')
    data15.append((Strategy_returns_table[Strategy_returns_table['Number of bins / Ranking Categories']==h]['Strategy Return Performance vs Benchmark Return'].str.rstrip('%').astype(float).mean())/100)
    data16.append(round(Strategy_returns_table[Strategy_returns_table['Number of bins / Ranking Categories']==h]['Long Strategy Mean Annual Ret'].mean(),5))
    data17.append(round(Strategy_returns_table[Strategy_returns_table['Number of bins / Ranking Categories']==h]['Long Strategy Standard Deviation'].mean(),5))
    data18.append(round(Strategy_returns_table[Strategy_returns_table['Number of bins / Ranking Categories']==h]['Benchmark Annual Ret'].mean(),5))
    data19.append(round(Strategy_returns_table[Strategy_returns_table['Number of bins / Ranking Categories']==h]['Benchmark Standard Deviation Ret'].mean(),5))
    
    
    strategy_desc = pd.DataFrame(data14, columns=['Strategy Investing in firms with the lowest industries per firm'])        
    average_overperformance_per_bin_qty = pd.DataFrame(data15, columns=['Average Overperformance of Strategy in ' + str((investment_scenarios)/(len(Strategy_returns_table['Number of bins / Ranking Categories'].unique()))) + ' test Scenarios'])        
    average_overperformance_per_bin_qty['Average Overperformance of Strategy in ' + str((investment_scenarios)/(len(Strategy_returns_table['Number of bins / Ranking Categories'].unique()))) + ' test Scenarios'] = average_overperformance_per_bin_qty['Average Overperformance of Strategy in ' + str((investment_scenarios)/(len(Strategy_returns_table['Number of bins / Ranking Categories'].unique()))) + ' test Scenarios'].astype(float).map("{:.2%}".format)
    
    
    average_strategy_annual_return = pd.DataFrame(data16, columns=['Average Strategy Annualized Return'])
    average_strategy_annual_return['Average Strategy Annualized Return'] = average_strategy_annual_return['Average Strategy Annualized Return'].astype(float).map("{:.3%}".format)
    
    average_strategy_standard_deviation = pd.DataFrame(data17, columns=['Average Strategy Standard Deviation'])
    average_strategy_standard_deviation['Average Strategy Standard Deviation'] = average_strategy_standard_deviation['Average Strategy Standard Deviation'].astype(float).map("{:.3%}".format)
    

    average_benchmark_annual_return = pd.DataFrame(data18, columns=['Average Benchmark Annualized Return'])
    average_benchmark_annual_return['Average Benchmark Annualized Return'] = average_benchmark_annual_return['Average Benchmark Annualized Return'].astype(float).map("{:.3%}".format)
    
    average_benchmark_standard_deviation = pd.DataFrame(data19, columns=['Average Benchmark Standard Deviation'])
    average_benchmark_standard_deviation['Average Benchmark Standard Deviation'] = average_benchmark_standard_deviation['Average Benchmark Standard Deviation'].astype(float).map("{:.3%}".format)
    
    
    
    Total_average_returns_per_bin = pd.concat([ strategy_desc, average_overperformance_per_bin_qty, average_strategy_annual_return,  average_strategy_standard_deviation,average_benchmark_annual_return,average_benchmark_standard_deviation ], axis=1)
end_time = time.time()



start date  201812 end date  202312 inv period 5 date dif 500 k  0 bin 8 x= 0
start date  201811 end date  202311 inv period 5 date dif 500 k  0 bin 8 x= 1
start date  201810 end date  202310 inv period 5 date dif 500 k  0 bin 8 x= 2
start date  201809 end date  202309 inv period 5 date dif 500 k  0 bin 8 x= 3
start date  201808 end date  202308 inv period 5 date dif 500 k  0 bin 8 x= 4
start date  201807 end date  202307 inv period 5 date dif 500 k  0 bin 8 x= 5
start date  201806 end date  202306 inv period 5 date dif 500 k  0 bin 8 x= 6
start date  201805 end date  202305 inv period 5 date dif 500 k  0 bin 8 x= 7
start date  201804 end date  202304 inv period 5 date dif 500 k  0 bin 8 x= 8
start date  201803 end date  202303 inv period 5 date dif 500 k  0 bin 8 x= 9
start date  201802 end date  202302 inv period 5 date dif 500 k  0 bin 8 x= 10
start date  201801 end date  202301 inv period 5 date dif 500 k  0 bin 8 x= 11
start date  200812 end date  201312 inv period 5 date dif 500 

start date  193805 end date  194305 inv period 5 date dif 500 k  80 bin 8 x= 7
start date  193804 end date  194304 inv period 5 date dif 500 k  80 bin 8 x= 8
start date  193803 end date  194303 inv period 5 date dif 500 k  80 bin 8 x= 9
start date  193802 end date  194302 inv period 5 date dif 500 k  80 bin 8 x= 10
start date  193801 end date  194301 inv period 5 date dif 500 k  80 bin 8 x= 11
start date  192812 end date  193312 inv period 5 date dif 500 k  90 bin 8 x= 0
start date  192811 end date  193311 inv period 5 date dif 500 k  90 bin 8 x= 1
start date  192810 end date  193310 inv period 5 date dif 500 k  90 bin 8 x= 2
start date  192809 end date  193309 inv period 5 date dif 500 k  90 bin 8 x= 3
start date  192808 end date  193308 inv period 5 date dif 500 k  90 bin 8 x= 4
start date  192807 end date  193307 inv period 5 date dif 500 k  90 bin 8 x= 5
start date  192806 end date  193306 inv period 5 date dif 500 k  90 bin 8 x= 6
start date  192805 end date  193305 inv period 5 d

start date  194310 end date  195310 inv period 10 date dif 1000 k  70 bin 8 x= 2
start date  194309 end date  195309 inv period 10 date dif 1000 k  70 bin 8 x= 3
start date  194308 end date  195308 inv period 10 date dif 1000 k  70 bin 8 x= 4
start date  194307 end date  195307 inv period 10 date dif 1000 k  70 bin 8 x= 5
start date  194306 end date  195306 inv period 10 date dif 1000 k  70 bin 8 x= 6
start date  194305 end date  195305 inv period 10 date dif 1000 k  70 bin 8 x= 7
start date  194304 end date  195304 inv period 10 date dif 1000 k  70 bin 8 x= 8
start date  194303 end date  195303 inv period 10 date dif 1000 k  70 bin 8 x= 9
start date  194302 end date  195302 inv period 10 date dif 1000 k  70 bin 8 x= 10
start date  194301 end date  195301 inv period 10 date dif 1000 k  70 bin 8 x= 11
start date  193312 end date  194312 inv period 10 date dif 1000 k  80 bin 8 x= 0
start date  193311 end date  194311 inv period 10 date dif 1000 k  80 bin 8 x= 1
start date  193310 end dat

start date  194812 end date  195312 inv period 5 date dif 500 k  70 bin 9 x= 0
start date  194811 end date  195311 inv period 5 date dif 500 k  70 bin 9 x= 1
start date  194810 end date  195310 inv period 5 date dif 500 k  70 bin 9 x= 2
start date  194809 end date  195309 inv period 5 date dif 500 k  70 bin 9 x= 3
start date  194808 end date  195308 inv period 5 date dif 500 k  70 bin 9 x= 4
start date  194807 end date  195307 inv period 5 date dif 500 k  70 bin 9 x= 5
start date  194806 end date  195306 inv period 5 date dif 500 k  70 bin 9 x= 6
start date  194805 end date  195305 inv period 5 date dif 500 k  70 bin 9 x= 7
start date  194804 end date  195304 inv period 5 date dif 500 k  70 bin 9 x= 8
start date  194803 end date  195303 inv period 5 date dif 500 k  70 bin 9 x= 9
start date  194802 end date  195302 inv period 5 date dif 500 k  70 bin 9 x= 10
start date  194801 end date  195301 inv period 5 date dif 500 k  70 bin 9 x= 11
start date  193812 end date  194312 inv period 5 d

start date  196306 end date  197306 inv period 10 date dif 1000 k  50 bin 9 x= 6
start date  196305 end date  197305 inv period 10 date dif 1000 k  50 bin 9 x= 7
start date  196304 end date  197304 inv period 10 date dif 1000 k  50 bin 9 x= 8
start date  196303 end date  197303 inv period 10 date dif 1000 k  50 bin 9 x= 9
start date  196302 end date  197302 inv period 10 date dif 1000 k  50 bin 9 x= 10
start date  196301 end date  197301 inv period 10 date dif 1000 k  50 bin 9 x= 11
start date  195312 end date  196312 inv period 10 date dif 1000 k  60 bin 9 x= 0
start date  195311 end date  196311 inv period 10 date dif 1000 k  60 bin 9 x= 1
start date  195310 end date  196310 inv period 10 date dif 1000 k  60 bin 9 x= 2
start date  195309 end date  196309 inv period 10 date dif 1000 k  60 bin 9 x= 3
start date  195308 end date  196308 inv period 10 date dif 1000 k  60 bin 9 x= 4
start date  195307 end date  196307 inv period 10 date dif 1000 k  60 bin 9 x= 5
start date  195306 end dat

In [169]:
print('Investment Scenarios: ' + str(investment_scenarios)  + ' - Run Time: ' , round(end_time - start_time , 2) , ' seconds')

Investment Scenarios: 456 - Run Time:  72.28  seconds


In [170]:
Strategy_returns_table

,Number of bins / Ranking Categories,Investment Start Date,Investment End Date,Investment Period in Years,Strategy Return Performance vs Benchmark Return,Strategy Total Cummulative Return,Long Strategy Mean Annual Ret,Long Strategy Standard Deviation,Long Strategy Return Risk Ratio,BenchMark Total Cummulative Return,Benchmark Annual Ret,Benchmark Standard Deviation Ret,Benchmark Return Risk Ratio
0,8,2018-12-01,2023-12-01,5,13.35%,1.737002,0.171144,0.180025,0.950670,1.532374,0.140503,0.204834,0.685934
1,8,2018-11-01,2023-11-01,5,13.35%,1.737002,0.171144,0.180025,0.950670,1.532374,0.140503,0.204834,0.685934
2,8,2018-10-01,2023-10-01,5,13.35%,1.737002,0.171144,0.180025,0.950670,1.532374,0.140503,0.204834,0.685934
3,8,2018-09-01,2023-09-01,5,13.35%,1.737002,0.171144,0.180025,0.950670,1.532374,0.140503,0.204834,0.685934
4,8,2018-08-01,2023-08-01,5,13.35%,1.737002,0.171144,0.180025,0.950670,1.532374,0.140503,0.204834,0.685934
...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,9,1933-05-01,1943-05-01,10,83.85%,4.442699,0.199002,0.281740,0.706331,2.416497,0.125374,0.248143,0.505249
452,9,1933-04-01,1943-04-01,10,81.86%,4.320346,0.197744,0.282980,0.698793,2.375652,0.124651,0.249248,0.500109
453,9,1933-03-01,1943-03-01,10,77.74%,3.922025,0.188546,0.282858,0.666576,2.206563,0.117490,0.249412,0.471068
454,9,1933-02-01,1943-02-01,10,81.82%,3.925401,0.190354,0.284099,0.670028,2.158980,0.116154,0.250521,0.463650


In [172]:
Total_average_returns_per_bin 

,Strategy Investing in firms with the lowest industries per firm,Average Overperformance of Strategy in 228.0 test Scenarios,Average Strategy Annualized Return,Average Strategy Standard Deviation,Average Benchmark Annualized Return,Average Benchmark Standard Deviation
0,Categorizing industries into 8 bins/categories,11.00%,12.890%,21.212%,11.459%,18.946%
1,Categorizing industries into 9 bins/categories,9.21%,12.522%,21.441%,11.459%,18.946%


In [173]:
Total_average_returns_per_bin['Average Overperformance of Strategy in ' + str((investment_scenarios)/(len(Strategy_returns_table['Number of bins / Ranking Categories'].unique()))) + ' test Scenarios'].max()

'9.21%'